In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard library imports.
import os
os.environ["DGLBACKEND"] = "pytorch"
import random
import itertools
import logging
import datetime
import collections

# Related third party imports.
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tqdm

import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn.metrics


# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
import src.logger as logger
import src.models.gnn as gnn
import src.models.hypervisor_placement as hypervisor_placement

logging.basicConfig(
    format="[%(funcName)30s()] %(message)s",
    level=logging.DEBUG,
    force=True,
)

# os.environ["PATH"] += os.pathsep + '/home/ubuntu/texlive/2022/bin/x86_64-linux'
# plt.style.use(['../custom-ieee.mplstyle'])


In [ ]:

networks = [('25_italy', 25), ('26_usa', 26), ('37_cost', 37),
            ('50_germany', 50)]
network_name, max_vSDN_size = networks[1]

simulation_group_id = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
simulation_group_folder = f"../results/{network_name}/static/tmp/{simulation_group_id}/"
os.mkdir(simulation_group_folder)

hp_possibilities = {
    'heu': ('heuristics', 'hypervisor count'),
    'heu_scs': ('heuristics', 'combined S CS'),
    'mc': ('heuristics', 'main controller'),
    'ilpk': ('ilp', 'hypervisor count'),
    'ilpa': ('ilp', 'acceptance ratio'),
    'ilpaf': ('ilp', 'acceptance and flexibility'),
}
static_type = 'ilpa'
hp_type, hp_objective = hp_possibilities[static_type]

hp_objective_list = (
    'acceptance_ratio',
    # 'hypervisor_load',
    # 'controller_load_request',
    # 'controller_load_switch',
)

hp_algo_settings = {
    'hp_type': [hp_type],
    'hp_objective': [hp_objective],
    'hp_objectives': [
        hp_objective_list
        # hp_obj for hp_obj in itertools.permutations(hp_objective_list)
        # if hp_obj[0] == 'acceptance_ratio'
    ],
    'candidate_selection': ['acceptance_ratio'],
    'n_extra_hypervisors': [0],
    'hypervisor_capacity': [None],
    'controller_capacity': [None],
    'n_diff_hypervisors': [0],
    'flexibility_weight': [None],
    'repeat': [50],
    'heuristic_randomness': [0.1],
}

simulation_settings = {
    'simulation_group_id': [simulation_group_id],
    'simulation_group_folder': [simulation_group_folder],
    'network_name': [network_name],
    'latency_factor': [0.5],
    'shortest_k': [16],
    'static_type': [static_type],
    'sim_repeat': [200],
    'max_request_size': [max(2, int(max_vSDN_size * 0.75))],
    'vSDN_count_ilp': [5, 10, 20, 50, 100, 200],
    'vSDN_size_ilp': [max(2, int(max_vSDN_size * 0.75))],
}

possible_settings = {**hp_algo_settings, **simulation_settings}
param_names_1 = list(possible_settings.keys())
setting_generator = [
    dict(zip(param_names_1, x))
    for x in itertools.product(*possible_settings.values())
]

possible_request_settings = {
    'request_size': np.arange(max_vSDN_size, 1, -1),
    'count': [100],
}

logging.info(f"\n'{simulation_group_folder}simulation-group-results.json'\n")

simulation_logs = []
dgl_graphs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    print()
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.run_multiple_static_simulation(
        possible_request_settings=possible_request_settings, generate_dgl=True, **setting)
    simulation_logs.extend(ns.get_logs())

    n_graphs = len(ns.dgl_graphs)
    dgl_graphs.extend(ns.dgl_graphs)

dgl_graph = dgl.batch(dgl_graphs)

# logger.save2json(simulation_group_folder + "simulation-group-results.json",
#                  simulation_logs)


In [ ]:
dgl.save_graphs("../data/processed/dgl_graphs/25_italy_600.bin", dgl_graph)

In [ ]:
glist, label_dict = dgl.load_graphs("../data/processed/dgl_graphs/25_italy_0.6_600.bin")
dgl_graph = glist[0]

In [ ]:
dgl_graph

In [ ]:
node_counts = collections.Counter()
for request in ns.vSDN_requests_ilp:
    node_counts.update(request.switches)

ordered_array = sorted(node_counts.items(), key=lambda x: x[0], reverse=False)
ordered_array = np.array([x[1] for x in ordered_array])

ordered_array = ordered_array / ordered_array.max()
print(ordered_array)


In [ ]:

hypervisor_vector = [1 if node in ns.network_operator.active_hypervisors else 0 for node in ns.network_operator.graph.nodes]

print(hypervisor_vector)


In [ ]:
dgl_graph.ndata['features'].shape

In [ ]:
gnn.assign_masks_to_graph(dgl_graph, train_ratio=0.8, val_ratio=0.1, shuffle=True)
features = dgl_graph.ndata['features']
labels = dgl_graph.ndata['label']
train_mask = dgl_graph.ndata['train_mask']
val_mask = dgl_graph.ndata['val_mask']
test_mask = dgl_graph.ndata['test_mask']

gnn_model, training_logs = gnn.train_gnn(
    dgl_graph,
    n_layers=4, n_hidden=100,
    lr=1e-2, weight_decay=5e-3,
    n_epochs=1001, weight=3.0
)

In [ ]:
training_logs['val_loss'][:10]

In [ ]:
# plot training loss

fig, ax = plt.subplots()
ax.plot(training_logs['loss'][:], label='Training loss')
ax.plot(training_logs['val_loss'][:], label='Validation loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_ylim(0, 1)
ax.legend();

# plt.savefig('../reports/figures/2024-/gnn-training-usa-2000.pdf')

In [ ]:
gnn.plot_roc(gnn_model, dgl_graph, features, labels, [train_mask, test_mask])

In [ ]:
gnn.plot_score_distribution(gnn_model, dgl_graph, features, labels);

In [ ]:
gnn.plot_precision_recall_curve(gnn_model, dgl_graph, features, labels)

In [ ]:
gnn.save_model(gnn_model, "../models/", 'gnn_model_italy_0.6_1000')

In [ ]:
gnn_model.parameter_dict

In [ ]:
gnn_model_2 = gnn.GCN(in_feats=8,
                    hid_feats=100,
                    out_feats=1,
                    n_layers=4)


gnn_model_2.load_state_dict(torch.load('../models/gnn_model_1.pt'))
gnn_model_2.eval()

In [ ]:
plot_score_distribution(gnn_model_2, dgl_graph, features, labels)

# Test on 1 graph

In [ ]:
import numpy as np
hypervisor_scores = hypervisor_placement.gnn_hypervisor_scoring(
    ns, **{
        'path': '../models/',
        'filename': 'gnn_model_1',
    }
)

# Assuming you have an array called 'array'
k = 5  # Number of largest items to select
largest_indices = np.argsort(hypervisor_scores)[-k:]
largest_items = hypervisor_scores[largest_indices]

print(largest_indices)
print(largest_items)


In [ ]:

test_graph = dgl_graphs[0]
test_features = test_graph.ndata['features']
test_labels = test_graph.ndata['label']

gnn_model.eval()
with torch.no_grad():
    logits = gnn_model(test_graph, test_features)
    print((logits>0.55).sum())

In [ ]:
plot_score_distribution(gnn_model, test_graph, test_features, test_labels)

In [ ]:
for test_graph in dgl_graphs:
    test_features = test_graph.ndata['features']
    test_labels = test_graph.ndata['label']

    gnn_model.eval()
    with torch.no_grad():
        logits = gnn_model(test_graph, test_features)
        print((logits>0.5).sum())

    # plor_score_distribution(gnn_model, test_graph, test_features, test_labels)

# Test

In [ ]:

networks = [('25_italy', 25), ('26_usa', 26), ('37_cost', 37),
            ('50_germany', 50)]
network_name, max_vSDN_size = networks[1]

simulation_group_id = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
simulation_group_folder = f"../results/{network_name}/static/tmp/{simulation_group_id}/"
os.mkdir(simulation_group_folder)

hp_possibilities = {
    'heu': ('heuristics', 'hypervisor count'),
    'heu_scs': ('heuristics', 'combined S CS'),
    'mc': ('heuristics', 'main controller'),
    'ilpk': ('ilp', 'hypervisor count'),
    'ilpa': ('ilp', 'acceptance ratio'),
    'ilpaf': ('ilp', 'acceptance and flexibility'),
    'gnn': ('gnn', 'acceptance ratio'),
}
static_type = 'gnn'
hp_type, hp_objective = hp_possibilities[static_type]

hp_objective_list = (
    'acceptance_ratio',
    # 'hypervisor_load',
    # 'controller_load_request',
    # 'controller_load_switch',
)

hp_algo_settings = {
    'hp_type': [hp_type],
    'hp_objective': [hp_objective],
    'hp_objectives': [
        hp_objective_list
        # hp_obj for hp_obj in itertools.permutations(hp_objective_list)
        # if hp_obj[0] == 'acceptance_ratio'
    ],
    'candidate_selection': ['acceptance_ratio'],
    'n_extra_hypervisors': [0],
    'hypervisor_capacity': [None],
    'controller_capacity': [None],
    'n_diff_hypervisors': [0],
    'flexibility_weight': [None],
    'repeat': [50],
    'heuristic_randomness': [0.1],
    'path2model': ['../models/'],
    'model_name': ['gnn_model_1'],
}

simulation_settings = {
    'simulation_group_id': [simulation_group_id],
    'simulation_group_folder': [simulation_group_folder],
    'network_name': [network_name],
    'latency_factor': [0.5],
    'shortest_k': [16],
    'static_type': [static_type],
    'sim_repeat': [10],
    'max_request_size': [max(2, int(max_vSDN_size * 0.75))],
    'vSDN_count_ilp': [100],
    'vSDN_size_ilp': [max(2, int(max_vSDN_size * 0.75))],
}

possible_settings = {**hp_algo_settings, **simulation_settings}
param_names_1 = list(possible_settings.keys())
setting_generator = [
    dict(zip(param_names_1, x))
    for x in itertools.product(*possible_settings.values())
]

possible_request_settings = {
    'request_size': np.arange(max_vSDN_size, 1, -1),
    'count': [400],
}

logging.info(f"\n'{simulation_group_folder}simulation-group-results.json'\n")

simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    print()
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.run_multiple_static_simulation(
        possible_request_settings=possible_request_settings, generate_dgl=False, **setting)
    simulation_logs.extend(ns.get_logs())

logger.save2json(simulation_group_folder + "simulation-group-results.json",
                 simulation_logs)
